In [1]:
import pandas as pd
import requests
import numpy as np
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [2]:
# place = "London, UK"
# G = ox.graph_from_place(place, network_type="bike")
# ox.save_graphml(G, filepath="../london_bike_network.graphml")

In [3]:
file_path = "../data/london_bike_network.graphml"
G = ox.load_graphml(file_path)

In [4]:
# visualize the map of London
print(G)

# fig, ax = ox.plot.plot_graph(G, bgcolor="white", node_color="red", edge_color="green", edge_linewidth=0.5, node_size=2)

MultiDiGraph with 327070 nodes and 746627 edges


#### Extract metrics/criteria/characteristics from OpenStreetMap for assessing bikeability

In [6]:
# extract all attributes from ndoe
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)
print(nodes.columns)

Index(['y', 'x', 'street_count', 'highway', 'junction', 'railway', 'ref',
       'geometry'],
      dtype='object')


In [7]:
street_count_list = []
railway_list = []
junction_node_list = []
highway_node_list = []

for node, data in G.nodes(data=True):
    street_count = data.get("street_count")
    railway = data.get("railway")
    highway = data.get('highway')
    junction = data.get('junction')

    street_count_list.append(street_count)
    railway_list.append(railway)
    junction_node_list.append(junction)
    highway_node_list.append(highway)

In [8]:
unique_street_count = set()
for item in street_count_list:
    if isinstance(item, list):  
        unique_street_count.update(item)
    elif item is not None:  
        unique_street_count.add(item)

print(f"Street count: {unique_street_count}")

Street count: {1, 2, 3, 4, 5, 6, 7, 8}


In [9]:
unique_railway = set()
for item in railway_list:
    if isinstance(item, list):  
        unique_railway.update(item)
    elif item is not None:  
        unique_railway.add(item)

print(f"railway types: {unique_railway}")

railway types: {'level_crossing', 'switch', 'tram_level_crossing', 'subway_entrance', 'crossing'}


- switch: A turnout is a node used for the bifurcation or merging of railway tracks (such as turning equipment)
- level_crossing: a location where railways and roads intersect at the same level, and it usually has railings or traffic lights
- tram_level_crossing: At the intersection of the tram and the road, the tram tracks cross the road on the same plane
- subway_entrance: The subway entrance refers to the entrance of a subway station, not necessarily the station itself. It is usually used for navigation or accessibility
- crossing: Railway pedestrian crossings or entrances may be used for pedestrians or non-motorized vehicles to cross the tracks

In [11]:
unique_junction_node = set()
for item in junction_node_list:
    if isinstance(item, list):  
        unique_junction_node.update(item)
    elif item is not None:  
        unique_junction_node.add(item)

print(f"junction types: {unique_junction_node}")

junction types: {'mini_roundabout', 'yes', 'Fulham Cross', 'roundabout', 'box', 'crossroads', 'intersection'}


- box: a marked area that prohibits parking at intersections and is used to prevent traffic congestion at intersections
- intersection: At ordinary intersections or crossroads, special shapes are generally not emphasized
- yes: General marking indicates "This is an intersection", but no specific type is specified
- roundabout: vehicles usually drive in a counterclockwise direction and there are yielding rules
- mini_roundabout: Small roundabout, often used in residential areas, may be a circle drawn on the ground rather than a solid structure
- Fulham Cross: a place name
- crossroads: 

In [13]:
unique_highway_node = set()
for item in highway_node_list:
    if isinstance(item, list):  
        unique_highway_node.update(item)
    elif item is not None:  
        unique_highway_node.add(item)

print(f"highway types: {unique_highway_node}")

highway types: {'mini_roundabout', 'turning_circle', 'traffic_signals;crossing', 'elevator', 'turning_loop', 'stop', 'motorway_junction', 'give_way', 'speed_camera', 'street_lamp', 'crossing', 'traffic_signals'}


- give_way: a sign indicating that one must yield at an intersection
- turning_loop: The rotary ring, a more complex U-turn structure, is commonly found at the end of public transportation or large lanes

#### Extract attributes from edge

In [16]:
# extract all attributes from edge
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
print(edges.columns)

Index(['osmid', 'access', 'highway', 'maxspeed', 'name', 'oneway', 'reversed',
       'length', 'geometry', 'lanes', 'ref', 'bridge', 'service', 'junction',
       'tunnel', 'width', 'est_width', 'area'],
      dtype='object')


In [17]:
# list for each attribute
access_list = []
highway_list = []
maxspeed_list = []
geometry_list = []       
lanes_list = []    
bridge_list = []     
service_list = []      
junction_list = []       
tunnel_list = []     
width_list = []    
est_width_list = []        
area_list = []   
ref_list = []  

In [18]:
for u, v, data in G.edges(data=True):
    access = data.get("access")         
    highway = data.get("highway")       
    maxspeed = data.get("maxspeed")                 
    oneway = data.get("oneway")         
    reversed_rd = data.get("reversed")  
    geometry = data.get("geometry")             
    lanes = data.get("lanes")                 
    bridge = data.get("bridge")                
    service = data.get("service")               
    junction = data.get("junction")             
    tunnel = data.get("tunnel")                
    width = data.get("width")                
    est_width = data.get("est_width")                  
    area = data.get("area")             
    ref = data.get("ref")               
    
    access_list.append(access)
    highway_list.append(highway)
    maxspeed_list.append(maxspeed)
    geometry_list.append(geometry)       
    lanes_list.append(lanes)    
    bridge_list.append(bridge)     
    service_list.append(service)      
    junction_list.append(junction)       
    tunnel_list.append(tunnel)     
    width_list.append(width)    
    est_width_list.append(est_width)        
    area_list.append(area)   
    ref_list.append(ref) 

In [19]:
unique_access = set()
for item in access_list:
    if isinstance(item, list):  
        unique_access.update(item)
    elif item is not None: 
        unique_access.add(item)

print(f"Access types: {unique_access}")

Access types: {'yes', 'delivery', 'permit', 'destination', 'staff', 'no', 'customers', 'construction', 'residents', 'permissive', 'restricted', 'taxi', 'emergency', 'unknown', 'agricultural', 'designated'}


- staff: Access restricted to staff members only.
- delivery: Access allowed for delivery vehicles.
- agricultural: Intended for agricultural use, such as farm vehicles.
- residents: Access restricted to local residents.
- emergency: Only emergency vehicles (e.g., ambulance, fire truck) are allowed.
- taxi: Access permitted for taxis.
- permissive: Access is allowed by informal or revocable permission, not a legal right.
- no: Access is prohibited.
- unknown: Access conditions are unknown.
- yes: Access is allowed for the general public.
- designated: Specifically designated for certain users or vehicles (e.g., bicycles, pedestrians).
- destination: Only vehicles with a destination on that road are allowed; through traffic is restricted.
- restricted: Access is restricted and may require special permission.
- permit: A formal permit is required for access.
- construction: Access is limited to construction vehicles or only during construction periods.
- customers: Access is allowed for customers of a shop or service facility.

In [21]:
# Road type
unique_highway = set()
for item in highway_list:
    if isinstance(item, list): 
        unique_highway.update(item)
    elif item is not None:  
        unique_highway.add(item)

print(f"Highway types: {unique_highway}")

Highway types: {'trunk_link', 'secondary', 'trunk', 'disused', 'tertiary', 'secondary_link', 'road', 'track', 'primary', 'service', 'primary_link', 'path', 'living_street', 'services', 'cycleway', 'bridleway', 'unclassified', 'pedestrian', 'crossing', 'residential', 'tertiary_link', 'busway'}


#### Main Road Types
- motorway: Highways for motor vehicles only; high-speed, long-distance roads.
- trunk: Major roads connecting cities; slightly lower in rank than motorways.
- primary: Primary roads serving as key routes within or between cities or regions.
- secondary: Secondary roads linking smaller towns or areas; lower than primary.
- tertiary: Tertiary roads that connect local roads to major ones; serve local traffic.
- unclassified: Minor local roads that don't fit other categories like residential or service.
- living_street: Shared space for vehicles and pedestrians, often with low speed limits; typically in residential zones.

#### Link Roads and Service Roads
- motorway_link / trunk_link / primary_link / secondary_link / tertiary_link: Ramps or connectors between main and secondary roads.
- service: Roads for services such as parking lots, industrial zones, or private access.
- services: Similar to service; may be a regional naming variation.

#### Non-motorized & Special Use Roads
- cycleway: Bicycle-only roads or lanes, often separated from motor traffic.
- path: General-purpose paths for pedestrians, cyclists, and sometimes horses; common in rural or park areas.
- bridleway: Horseback riding trails; usually closed to motor vehicles.
- pedestrian: Pedestrian-only streets, often in commercial areas or parks.
- track: Unpaved or dirt roads used for farming or forestry access.
- residential: Roads within residential areas, usually with low traffic.

#### Other Types
- crossing: Crosswalks or pedestrian crossings.
- busway: Bus-only lanes or roads restricted to public transport.
- disused: Abandoned roads no longer in official use.
- Let me know if you'd like this in Markdown table format for documentation or presentation.

In [23]:
unique_maxspeed = set()
for item in maxspeed_list:
    if isinstance(item, list):  
        unique_maxspeed.update(item)
    elif item is not None:  
        unique_maxspeed.add(item)

print(f"{unique_maxspeed}")

{'50 mph', '3 mph', '70 mph', '10 mph', '64', '4 mph', 'walk', '40 mph', '60 mph', '5', '12 mph', '15 mph', '7', '5 mph', '20 mph', '30 mph'}


In [24]:
unique_lanes = set()
for item in lanes_list:
    if isinstance(item, list):
        unique_lanes.update(item)
    elif item is not None:  
        unique_lanes.add(item)
        
print(unique_lanes)

{'0', '20', '4', '2', '1.5', '3', '8', '7', '5', '1', '6'}


In [25]:
unique_bridge = set()
for item in bridge_list:
    if isinstance(item, list):
        unique_bridge.update(item)
    elif item is not None:  
        unique_bridge.add(item)
        
print(unique_bridge)

{'yes', 'viaduct', 'pier', 'boardwalk', 'movable'}


- viaduct: A viaduct, typically a long elevated bridge structure used to span valleys, rivers, or urban obstacles. Usually longer than a standard bridge.
- yes: Indicates that the way is a bridge, but without specifying the type.
- pier: A pier bridge, often extending into water, used for docking boats or pedestrian access.
- boardwalk: A wooden walkway, often elevated, commonly found in wetlands, forests, or coastal areas.
- movable: A movable bridge that can open, lift, or rotate to allow ships to pass—e.g., drawbridges or swing bridges.

In [27]:
unique_service = set()
for item in service_list:
    if isinstance(item, list):
        unique_service.update(item)
    elif item is not None:  
        unique_service.add(item)
        
print(unique_service)

{'delivery', 'drive', 'drive-through', 'alley', 'garages', 'Service_Road', 'bus', 'road', 'bus_station', 'taxi', 'driveway', 'parking', 'ar', 'access', 'alleyway', 'yard', 'slipway', 'emergency', 'psv', 'parking_aisle', 'emergency_access', 'residential', 'layby', 'unknown', 'fuel', 'ramp'}


#### Residential & General Service Roads
- residential: Roads within residential areas.
- road: A generic road; often used as a placeholder. Not recommended for permanent tagging.

#### Traffic-Related Service Roads
- driveway: Private driveways, usually connecting homes or buildings to the main road.
- drive: Similar to driveway, possibly used regionally.
- parking: Roads or lanes within parking areas.
- parking_aisle: Lanes inside parking lots for vehicle circulation.
- bus_station: Roads or platforms inside a bus station.
- bus: May indicate bus-related lanes or internal roads.

#### Logistics & Commercial Use
- delivery: Roads specifically for goods delivery.
- drive-through: Roads that serve drive-through businesses (e.g., fast food, banks).
- fuel: Roads within fuel stations.
- garages: Internal roads in garages or vehicle repair areas.

#### Alleys & Narrow Roads
- alley / alleyway: Narrow urban lanes, often behind buildings or used for pedestrian access.
- yard: Roads inside industrial yards or warehouse areas.
- layby: Lay-bys or emergency pull-off areas on roads.

#### Emergency & Special-Purpose Roads
- emergency_access / emergency: Roads restricted to emergency services (e.g., ambulance, fire trucks).
- ramp: Inclined roads or ramps connecting roads of different elevation.
- slipway: Boat ramps used for launching or retrieving vessels.

#### Public Transport-Related
- taxi: Taxi-only lanes or parking areas.
- psv (Public Service Vehicle): Lanes for buses or other public service vehicles.

#### Other / Miscellaneous
- access: General-purpose roads, possibly with access restrictions.
- Service_Road: Broad term for service roads, often adjacent to highways or in industrial zones.
- unknown: Road type is not clearly specified.

In [29]:
unique_junction = set()
for item in junction_list:
    if isinstance(item, list):
        unique_junction.update(item)
    elif item is not None:  
        unique_junction.add(item)
        
print(unique_junction)

{'roundabout', 'turning_loop', 'teardrop', 'gyratory', 'circular', 'intersection', 'approach', 'jughandle'}


#### Traditional Intersections
- intersection: A standard at-grade junction where two or more roads cross each other.
- approach: Typically refers to the segment of road leading up to an intersection; exact meaning may depend on context.

#### Roundabout-Type Intersections
- roundabout: A conventional circular intersection where vehicles travel in one direction around a central island and exit at their chosen point.
- circular: Similar to a roundabout but may allow freer or less regulated entry and exit movements.
- gyratory: A large, often multi-lane roundabout system, sometimes signal-controlled; more complex than a standard roundabout.

#### Special Intersection Types
- jughandle: A ramp-style intersection where drivers exit the main road before turning left or making a U-turn—designed to reduce direct left turns.
- turning_loop: A loop at the end of a dead-end road or cul-de-sac that allows vehicles to turn around.
- teardrop: A teardrop-shaped intersection design, often used in high-traffic areas to provide smoother traffic flow, especially near ramps or interchanges.

In [31]:
unique_tunnel = set()
for item in tunnel_list:
    if isinstance(item, list):
        unique_tunnel.update(item)
    elif item is not None:  
        unique_tunnel.add(item)
        
print(unique_tunnel)

{'yes', 'covered', 'passage', 'no', 'building_passage'}


#### Basic Tunnel Indicators
- yes: Indicates the road is a standard tunnel, typically passing through mountains, underground, or other enclosed spaces.
- no: Indicates the road is not a tunnel.

#### Special Tunnel or Passage Types
- passage: A general passage, often a smaller tunnel for pedestrians or cyclists rather than a full-scale road tunnel.
- building_passage: A passage through or under a building, such as walkways or roads that pass beneath urban structures.
- covered: A covered road—not a traditional tunnel, but one that is enclosed from above by a building or structure.

In [33]:
unique_width = set()
for item in width_list:
    if isinstance(item, list):
        unique_width.update(item)
    elif item is not None:  
        unique_width.add(item)
        
print(unique_width)

{'3.9', '1m', '3.4', '2.0', '6.6', '8.9', '0.7', '12.5', '6.2', '0.9', '0', '6.8', '1.25', '0.7 m', '1.5', '13', '11.1', '3.2', '7', '2.8', '2.17', '8.2', '1.6', '2.2', '5.8', '9', '10.6', '6 m', '0.5', '4.3', '5.7', '5.5', '0.8 m', '10.4', '4.5', '9.5', '11.7', '12.6', '3', '2.4', '7.4', '2.25', '10.5', '7.5', '6.3', '5.2', '5 m', '6.5', '0.3', '5.9', '0.6', '12.4', '15.4', '5', '4.7', '3.3', '4.4', '5.6', '8.6', '5.1', '1 m', '9.9', '8.8', '2.7', '1.5 m', '8.1', '0.3 m', '2.6', '10 m', '2.13', '8.3', '6.1', '8.4', '14', '1.9', '6 ft 6', '14.6', '11', '6.7', '4.6', '2', '18', '7.7', '7.9', '1.2', '12.1', '1.1', '6\'6"', '6.9', '3.8', '7.2', '0.8', '4.9', '2.3', '5.4', '1.75', '4 m', '1.4', '10.7', '8.5', '9.1', '1.0', '2.5', '2m', '10', '3.0', '4.8', '8.7', '0.4 m', '0.2', '10.8', '3.6', '0.2 m', '4.1', '15', '3.75', '11.4', '1', '3.1', '2 m', '6.4', '6', '9.3', '7.3', '1.7', '9.4', '7.8', '7.25', '10.2', '9.2', '4', '7.1', '5.3', '2.5 m', '0.5 m', '21', '3.5', '11.9', '2.0 m', '0.3m'

In [34]:
unique_est_width = set()
for item in est_width_list:
    if isinstance(item, list):
        unique_est_width.update(item)
    elif item is not None:  
        unique_est_width.add(item)
        
print(unique_est_width)

{'20', '2', '5', '1', '6', '1.25', '1.5', '3.5m', '1.75', '4', '5ft', '3.5', '2.5', '4m', '2.5m', '8.5m', '2m', '4.5m', '4.5', 'est_width', '3', '5m'}


In [35]:
unique_area = set()
for item in area_list:
    if isinstance(item, list):
        unique_area.update(item)
    elif item is not None:  
        unique_area.add(item)
        
print(unique_area)

{'no'}


In [36]:
unique_ref = set()
for item in ref_list:
    if isinstance(item, list):
        unique_ref.update(item)
    elif item is not None:  
        unique_ref.add(item)
        
print(list(unique_ref)[:20])

['B515', 'A23', 'A1261', 'FP106', 'A11', 'B137', 'B160', 'S115', 'A127', 'B168', 'B276', 'RN4', 'A109', 'B273', 'B305', 'B234', 'A12', 'A479', 'B310', 'A1003']


In [37]:
print(geometry_list[:5])

[None, None, <LINESTRING (-0.1 51.527, -0.1 51.527, -0.1 51.528, -0.1 51.528)>, None, <LINESTRING (-0.2 51.524, -0.2 51.524, -0.2 51.524)>]


#### XML dataset ouput

In [40]:
# Requesting an API for XML data
url = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"
response = requests.get(url)

# parse XML
root = ET.fromstring(response.content)

data = []
# Read information about shared bike stations
for station in root.findall('station'):
    station_id = station.find('id').text
    station_name = station.find('name').text
    lat = station.find('lat').text
    lng = station.find('long').text
    installed = station.find('installed').text
    locked = station.find('locked').text
    tempStation = station.find('temporary').text
    bikes_available = station.find('nbBikes').text
    standardBikes = station.find('nbStandardBikes').text
    eBikes = station.find('nbEBikes').text
    docks_available = station.find('nbEmptyDocks').text
    total_docks = station.find('nbDocks').text

    data.append([station_id, station_name, lat, lng, installed, locked, tempStation, bikes_available, standardBikes, eBikes, docks_available, total_docks])


df = pd.DataFrame(data, columns=["ID", "Station", "Latitude", "Longitude", "Installed", "locked", "Temporary Station", "Available Bikes", "Standard Bikes", "E-Bikes", "Empty Docks", "Total Docks"])
df.head()

,ID,Station,Latitude,Longitude,Installed,locked,Temporary Station,Available Bikes,Standard Bikes,E-Bikes,Empty Docks,Total Docks
0,1,"River Street , Clerkenwell",51.52916347,-0.109970527,true,false,false,5,4,1,12,19
1,2,"Phillimore Gardens, Kensington",51.49960695,-0.197574246,true,false,false,9,7,2,27,37
2,3,"Christopher Street, Liverpool Street",51.52128377,-0.084605692,true,false,false,0,0,0,31,32
3,4,"St. Chad's Street, King's Cross",51.53005939,-0.120973687,true,false,false,22,18,4,1,23
4,5,"Sedding Street, Sloane Square",51.49313,-0.156876,true,false,false,15,15,0,8,27
